In [3]:
import torch 
from collections.abc import Iterable
from datasets import load_dataset, list_datasets

# Model and tokenizer from 🤗 Transformers
from transformers import AutoModelForSequenceClassification, \
    BertForSequenceClassification, BertTokenizerFast, AutoModel, AutoTokenizer

# Code you will write for this assignment
from train_model import init_model, preprocess_dataset, init_trainer
from test_model import init_tester

In [4]:
hate_speech = load_dataset("hate_speech_offensive")
split = hate_speech["train"].train_test_split(.2, seed=3463)
hate_speech["train"] = split["train"]
hate_speech["test"] = split["test"]

split = hate_speech["train"].train_test_split(.2, seed=3463)
hate_speech["train"] = split["train"]
hate_speech["val"] = split["test"]

Found cached dataset hate_speech_offensive (/Users/jiayuansong/.cache/huggingface/datasets/hate_speech_offensive/default/1.0.0/5f5dfc7b42b5c650fe30a8c49df90b7dbb9c7a4b3fe43ae2e66fabfea35113f5)
100%|██████████| 1/1 [00:00<00:00, 22.04it/s]
Loading cached split indices for dataset at /Users/jiayuansong/.cache/huggingface/datasets/hate_speech_offensive/default/1.0.0/5f5dfc7b42b5c650fe30a8c49df90b7dbb9c7a4b3fe43ae2e66fabfea35113f5/cache-e8bc8b2108a4c2d9.arrow and /Users/jiayuansong/.cache/huggingface/datasets/hate_speech_offensive/default/1.0.0/5f5dfc7b42b5c650fe30a8c49df90b7dbb9c7a4b3fe43ae2e66fabfea35113f5/cache-39dae8e28af0c6f7.arrow
Loading cached split indices for dataset at /Users/jiayuansong/.cache/huggingface/datasets/hate_speech_offensive/default/1.0.0/5f5dfc7b42b5c650fe30a8c49df90b7dbb9c7a4b3fe43ae2e66fabfea35113f5/cache-1abefe43e27c4edd.arrow and /Users/jiayuansong/.cache/huggingface/datasets/hate_speech_offensive/default/1.0.0/5f5dfc7b42b5c650fe30a8c49df90b7dbb9c7a4b3fe43ae2e66

In [5]:
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base")

emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
hate_speech["train"] = preprocess_dataset(hate_speech["train"], tokenizer)
hate_speech["val"] = preprocess_dataset(hate_speech["val"], tokenizer)
hate_speech["test"] = preprocess_dataset(hate_speech["test"], tokenizer)

# Visualize the preprocessed dataset
for k, v in hate_speech["val"][:2].items(): 
    print("{}:\n{}\n{}\n".format(k, type(v),
                                 [item[:20] if isinstance(item, Iterable) else 
                                 item for item in v[:5]]))

Loading cached processed dataset at /Users/jiayuansong/.cache/huggingface/datasets/hate_speech_offensive/default/1.0.0/5f5dfc7b42b5c650fe30a8c49df90b7dbb9c7a4b3fe43ae2e66fabfea35113f5/cache-c5ff1d17dbe917a1.arrow
Loading cached processed dataset at /Users/jiayuansong/.cache/huggingface/datasets/hate_speech_offensive/default/1.0.0/5f5dfc7b42b5c650fe30a8c49df90b7dbb9c7a4b3fe43ae2e66fabfea35113f5/cache-84813d364895bf57.arrow
Loading cached processed dataset at /Users/jiayuansong/.cache/huggingface/datasets/hate_speech_offensive/default/1.0.0/5f5dfc7b42b5c650fe30a8c49df90b7dbb9c7a4b3fe43ae2e66fabfea35113f5/cache-9bfc15eafb4812d9.arrow
Loading cached processed dataset at /Users/jiayuansong/.cache/huggingface/datasets/hate_speech_offensive/default/1.0.0/5f5dfc7b42b5c650fe30a8c49df90b7dbb9c7a4b3fe43ae2e66fabfea35113f5/cache-7b3c62595437f215.arrow
Loading cached processed dataset at /Users/jiayuansong/.cache/huggingface/datasets/hate_speech_offensive/default/1.0.0/5f5dfc7b42b5c650fe30a8c49df90

count:
<class 'list'>
[6, 3]

hate_speech_count:
<class 'list'>
[2, 0]

offensive_language_count:
<class 'list'>
[4, 3]

neither_count:
<class 'list'>
[0, 0]

class:
<class 'list'>
[1, 1]

tweet:
<class 'list'>
['RT @realist_iLLest: ', 'Almost done with thi']

demo_props:
<class 'list'>
[[0.26882176380724737, 0.36224836621752143, 0.0058887965306269295, 0.36304107344460435], [0.538484164995624, 0.22799105326083585, 0.005798052499449793, 0.22772672924409032]]

labels:
<class 'list'>
[1, 1]

input_ids:
<class 'list'>
[[0, 246, 5238, 3320, 10987, 524, 510, 7310, 2110, 22, 19295, 33123, 1981, 17742, 5238, 44662, 864, 31959, 22, 2443], [0, 4267, 270, 30, 33, 397, 245, 551, 639, 28, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1]]

token_type_ids:
<class 'list'>
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

attention_mask:
<class 'list'>
[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [10]:
import numpy as np

In [34]:
data = hate_speech['test'] 
props = np.array(data['demo_props']) 
aae_idx = np.where(props[:,0] >= 0.6)
aae_idx[0] 

(1531,)

In [63]:
import pickle  
import numpy as np
from datasets import Dataset

class Evaluation:
    
    def __init__(self, file: str, data: Dataset, threshold=0.6): 
        ''' 
        Parameters: 
            
        '''
        self.predictions = self.parse_result(file)  
        self.labels = self.parse_label(data)
        self._aae_idx, self._white_idx = self.parse_demo(data, threshold)
        assert len(self.labels) == len(self.predictions) 

    @ staticmethod 
    def parse_result(file): 
        with open(file, 'rb') as f:
            obj = pickle.load(f)
        return obj.predictions.argmax(axis=1) 
    
    @ staticmethod 
    def parse_demo(data, threshold): 
        props = np.array(data['demo_props']) 
        aae = np.where(props[:,0] >= threshold)[0]
        # white = np.where(props[:,-1] >= threshold)[0] 
        white = np.where(props[:,0] < threshold)[0] # non-aae 
        return aae, white 
                
    @ staticmethod 
    def parse_label(data):
        return np.array(data['labels']) 
    
    def get_EOD(self): 
        ''' 
        returns the 'equal opportunity difference' {
            between true positive rates of white and aae predictions; 
            the closer to 0 the better; 
        } 
        ''' 
        
        white_tpr = (self.predictions[self._white_pos_true] == 1).sum() / self._white_pos_true.shape[0] 
        aae_tpr = (self.predictions[self._aae_pos_true] == 1).sum() / self._aae_pos_true.shape[0]
        print(white_tpr) 
        print(aae_tpr)
        return white_tpr - aae_tpr 

    def get_AOD(self):
        pass 

    def get_SPD(self): 
        '''
        returns the 'statistical parity difference' {
            between probabilities of toxic white and aae classifications; 
            the closer to 0 the better;  

        }
        ''' 
        return  self._white_pos_true.shape[0]/self._white_idx.shape[0] - self._aae_pos_true.shape[0]/self._aae_idx.shape[0] 

    def get_DI(self): 
        '''
        returns the 'disparate impact' { 
            not impacted by annotations; 
            acceptable between 0.8 and 1.2; 
        }
        '''
        di_toxic = (self._aae_pos_true.shape[0]/self._aae_idx.shape[0]) / (self._white_pos_true.shape[0]/self._white_idx.shape[0])
        di_nontoxic = (1 - self._aae_pos_true.shape[0]/self._aae_idx.shape[0]) / (1 - self._white_pos_true.shape[0]/self._white_idx.shape[0]) 
        return (di_toxic, di_nontoxic)

In [76]:
predictions = Evaluation.parse_result('test_results.p')  
labels = Evaluation.parse_label(hate_speech['test'])
_aae_idx, _white_idx = Evaluation.parse_demo(hate_speech['test'], 0.1)

In [77]:
pred_white = predictions[_white_idx] 
pred_aae = predictions[_aae_idx] 

In [82]:
predictions[predictions==0]

array([0])

In [83]:
with open('test_results.p', 'rb') as f:
    obj = pickle.load(f)

In [85]:
obj.predictions.argmax(axis=-1)

array([1, 1, 2, ..., 1, 1, 1])

In [78]:
np.unique(pred_aae) 

array([1, 2])

In [58]:
eval = Evaluation('test_results.p', hate_speech['test'], 0.6) 

In [59]:
eval.get_EOD()

0.7914383561643835
0.7939086294416243


-0.0024702732772408087

In [60]:
eval.get_DI() 

(1.3133510933210728, 0.10645977295162863)

In [61]:
eval.get_SPD() 

-0.2319942171646887

In [18]:
import pickle

In [95]:
with open('test_results.p', 'rb') as f:
    obj = pickle.load(f)
predictions = obj.predictions.argmax(axis=1)  
predictions

array([1, 1, 2, ..., 1, 1, 1])

In [23]:
predictions[[1,2]]

array([1, 0])

In [8]:
import numpy as np

texts = np.array(hate_speech['test']['tweet'])
true_labels = np.array(hate_speech['test']['labels'])
len(true_labels)

7435

In [11]:
true_labels.shape

(7435,)

In [12]:
pred_labels.shape

(2479,)

In [10]:
texts[(true_labels == 0) & (pred_labels == 1)]

ValueError: operands could not be broadcast together with shapes (7435,) (2479,) 